## Wav파일 전처리

In [ ]:
import os
import subprocess
from pathlib import Path

def preprocess_wav_directory(input_dir, output_dir, sample_rate=16000):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for wav_file in input_dir.glob("*.wav"):
        output_file = output_dir / wav_file.name
        command = [
            "ffmpeg",
            "-y",  # overwrite without asking
            "-i", str(wav_file),
            "-ar", str(sample_rate),       # Resample to 16kHz
            "-ac", "1",                    # Convert to mono
            "-acodec", "pcm_s16le",        # 16-bit PCM
            str(output_file)
        ]
        subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print(f"✅ 전처리 완료: {len(list(output_dir.glob('*.wav')))}개 파일이 {output_dir}에 저장되었습니다.")

# 예시 사용
input_path = 'input_wav'
output_path = "output_wav"
preprocess_wav_directory(input_path,output_path )


## STT

In [ ]:
import os
from dotenv import load_dotenv
# .env 파일 로드
load_dotenv()

# 환경변수 가져오기
speech_key = os.getenv("speech_key")

In [ ]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_speech(audio_path: str, speech_key: str, region: str) -> str:
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=region)
    speech_config.speech_recognition_language = "ko-KR"  # ✅ 한국어 설정

    audio_config = speechsdk.audio.AudioConfig(filename=audio_path)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("🎙️ STT 실행 중...")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("📝 인식된 텍스트:", result.text)
        return result.text
    else:
        print("❌ 음성 인식 실패:", result.reason)
        return ""

# 예시 실행
region = "eastus"
prompt_audio_path = "output_wav/노인남여_노인대화07_F_1526682663_63_수도권_실내_08352.wav"

prompt_text = transcribe_speech(prompt_audio_path, speech_key, region)


## TTS

In [ ]:
from inference_module import init_engine, run_tts
import numpy as np
from scipy.io.wavfile import write as write_wav
engine = init_engine()

input_text = """
이 사진은 내가 스무 살 때, 처음 서울 구경 갔을 때 찍은 거야.
저기 옆에 있는 친구는 순이, 참 말도 많고 웃음도 많던 아이였지.
그날 남대문시장에서 산 하늘색 원피스를 아직도 기억해.
사진 속 나는 참 해맑은데, 그땐 세상이 다 설레고 신기했지.
요즘도 이 사진 보면 가끔 그 시절 냄새가 나는 것 같아.
"""
prompt_audio_path = "output_wav/노인남여_노인대화07_F_1526682663_63_수도권_실내_08352.wav"

# 3. 추론 실행
sample_rate, waveform = run_tts(engine, input_text, prompt_audio_path, prompt_text)

# 4. float32 → int16로 변환하여 저장
waveform_int16 = (waveform * 32767).astype(np.int16)
write_wav("model_output/8000/1.wav", sample_rate, waveform_int16)

print("✅ 생성 완료: output.wav")
